# GroupRareLevelsTransformer
This notebook shows the functionality in the `GroupRareLevelsTransformer` class. This transformer groups together infrequently occuring levels of a variables into new level labelled 'rare' (by default). <br>
Rare levels are determine by either the percent of rows or the percent of weight that falls into each category.

In [69]:
import pandas as pd
import numpy as np
from pprint import pprint
from sklearn.datasets import load_diabetes

In [70]:
import tubular
from tubular.nominal import GroupRareLevelsTransformer

In [71]:
tubular.__version__

'1.0.1'

## Load diabetes dataset from sklearn
We also create a categorical column from `bmi` and treat it as unordered for demonstration purposes in this notebook.

In [72]:
diabetes = load_diabetes(return_X_y=False, as_frame=True)["data"]

In [73]:
diabetes["bmi_cut"] = pd.cut(diabetes["bmi"], bins=20)

In [74]:
diabetes.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,bmi_cut
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,"(0.0532, 0.0662]"
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,"(-0.0642, -0.0512]"
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,"(0.0401, 0.0532]"
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,"(-0.012, 0.00102]"
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,"(-0.0381, -0.0251]"


In [75]:
diabetes["bmi_cut"].value_counts(dropna=False, normalize=True)

(-0.012, 0.00102]     0.117647
(-0.0381, -0.0251]    0.110860
(-0.0251, -0.012]     0.110860
(-0.0512, -0.0381]    0.085973
(0.00102, 0.0141]     0.085973
(0.0141, 0.0271]      0.081448
(-0.0642, -0.0512]    0.063348
(0.0532, 0.0662]      0.063348
(0.0401, 0.0532]      0.056561
(0.0271, 0.0401]      0.056561
(-0.0772, -0.0642]    0.049774
(0.0662, 0.0793]      0.036199
(-0.0905, -0.0772]    0.022624
(0.0923, 0.105]       0.020362
(0.0793, 0.0923]      0.015837
(0.118, 0.131]        0.009050
(0.105, 0.118]        0.006787
(0.158, 0.171]        0.004525
(0.131, 0.144]        0.002262
(0.144, 0.158]        0.000000
Name: bmi_cut, dtype: float64

## Simple usage

### Initialising GroupRareLevelsTransformer
The user must set `cut_off_percent` to determine rare levels. <br>
Note, multiple columns to group can be specified in the `columns` argument and they will all use the same cut off point.

In [76]:
grp_1 = GroupRareLevelsTransformer(
    columns=["bmi_cut"], cut_off_percent=0.10, copy=True, verbose=True
)

BaseTransformer.__init__() called


### GroupRareLevelsTransformer fit
The `fit` method determines the 'non-rare' levels from the input data, it must be run before the `transform` method. <br>
The 'non-rare' levels are stored in the `mapping_` attribute.

In [77]:
grp_1.fit(diabetes)

BaseTransformer.fit() called


GroupRareLevelsTransformer(columns=['bmi_cut'], cut_off_percent=0.1)

In [78]:
pprint(grp_1.mapping_)

{'bmi_cut': [Interval(-0.012, 0.00102, closed='right'),
             Interval(-0.0251, -0.012, closed='right'),
             Interval(-0.0381, -0.0251, closed='right')]}


### GroupRareLevelsTransformer transform
The `transform` method maps any levels that are not present in the `mapping_` dict to 'rare'.

In [79]:
diabetes_2 = grp_1.transform(diabetes)

BaseTransformer.transform() called


In [80]:
diabetes_2["bmi_cut"].value_counts(dropna=False)

rare                  292
(-0.012, 0.00102]      52
(-0.0381, -0.0251]     49
(-0.0251, -0.012]      49
(0.0662, 0.0793]        0
(0.158, 0.171]          0
(0.144, 0.158]          0
(0.131, 0.144]          0
(0.118, 0.131]          0
(0.105, 0.118]          0
(0.0923, 0.105]         0
(0.0793, 0.0923]        0
(-0.0905, -0.0772]      0
(0.0532, 0.0662]        0
(-0.0772, -0.0642]      0
(0.0271, 0.0401]        0
(0.0141, 0.0271]        0
(0.00102, 0.0141]       0
(-0.0512, -0.0381]      0
(-0.0642, -0.0512]      0
(0.0401, 0.0532]        0
Name: bmi_cut, dtype: int64

## Changing the rare level label
The name of the rare level can be changed by using the `rare_level_name` argument when initialising the GroupRareLevelsTransformer object.

In [81]:
grp_2 = GroupRareLevelsTransformer(
    columns=["bmi_cut"],
    rare_level_name="zzz",
    cut_off_percent=0.10,
    copy=True,
    verbose=False,
)

In [82]:
grp_2.fit(diabetes)

GroupRareLevelsTransformer(columns=['bmi_cut'], cut_off_percent=0.1,
                           rare_level_name='zzz')

In [83]:
diabetes_3 = grp_2.transform(diabetes)

In [84]:
diabetes_3["bmi_cut"].value_counts(dropna=False)

zzz                   292
(-0.012, 0.00102]      52
(-0.0381, -0.0251]     49
(-0.0251, -0.012]      49
(0.0662, 0.0793]        0
(0.158, 0.171]          0
(0.144, 0.158]          0
(0.131, 0.144]          0
(0.118, 0.131]          0
(0.105, 0.118]          0
(0.0923, 0.105]         0
(0.0793, 0.0923]        0
(-0.0905, -0.0772]      0
(0.0532, 0.0662]        0
(-0.0772, -0.0642]      0
(0.0271, 0.0401]        0
(0.0141, 0.0271]        0
(0.00102, 0.0141]       0
(-0.0512, -0.0381]      0
(-0.0642, -0.0512]      0
(0.0401, 0.0532]        0
Name: bmi_cut, dtype: int64

## Recording rare levels
By default the levels identified as 'rare' i.e. those that fall below the `cut_off_percent` value are not recorded. This is because there could be potentially a large number of levels. This can be changed by setting the `record_rare_levels` argument to `True` when initialising the GroupRareLevelsTransformer object. <br>
If this is the case the rare levels are recorded in a dict in the `rare_levels_record_` attribute on the transformer. <br>
The user should take care doing this if using the transformer on columns with many levels as this can result in a large transformer object. 

In [85]:
grp_3 = GroupRareLevelsTransformer(
    columns=["bmi_cut"],
    record_rare_levels=True,
    cut_off_percent=0.10,
    copy=True,
    verbose=False,
)

In [86]:
grp_3.fit(diabetes)

GroupRareLevelsTransformer(columns=['bmi_cut'], cut_off_percent=0.1)

In [87]:
pprint(grp_3.rare_levels_record_)

{'bmi_cut': [Interval(-0.0512, -0.0381, closed='right'),
             Interval(-0.0642, -0.0512, closed='right'),
             Interval(-0.0772, -0.0642, closed='right'),
             Interval(-0.0905, -0.0772, closed='right'),
             Interval(0.00102, 0.0141, closed='right'),
             Interval(0.0141, 0.0271, closed='right'),
             Interval(0.0271, 0.0401, closed='right'),
             Interval(0.0401, 0.0532, closed='right'),
             Interval(0.0532, 0.0662, closed='right'),
             Interval(0.0662, 0.0793, closed='right'),
             Interval(0.0793, 0.0923, closed='right'),
             Interval(0.0923, 0.105, closed='right'),
             Interval(0.105, 0.118, closed='right'),
             Interval(0.118, 0.131, closed='right'),
             Interval(0.131, 0.144, closed='right'),
             Interval(0.144, 0.158, closed='right'),
             Interval(0.158, 0.171, closed='right')]}


## Using row by row weights to identify rare levels
If records in the data do not have equal weight the user can set the `weight` argument when initialising the GroupRareLevelsTransformer object so `cut_off_percent` applies to the sum of weight rather than sum of rows. <br>
In this example we create a dummy weights column and set a rows with negative `bmi` values (note, columns have been standardised in this dataset) with a high weight compared to positive `bmi` values.

In [88]:
diabetes["weights"] = 1
diabetes.loc[diabetes["bmi"] < 0, "weights"] = 10

In [89]:
grp_4 = GroupRareLevelsTransformer(
    columns=["bmi_cut"],
    weight="weights",
    cut_off_percent=0.10,
    copy=True,
    verbose=False,
)

In [90]:
grp_4.fit(diabetes)

GroupRareLevelsTransformer(columns=['bmi_cut'], cut_off_percent=0.1,
                           weight='weights')

In [91]:
pprint(grp_4.mapping_)

{'bmi_cut': [Interval(-0.012, 0.00102, closed='right'),
             Interval(-0.0251, -0.012, closed='right'),
             Interval(-0.0381, -0.0251, closed='right'),
             Interval(-0.0512, -0.0381, closed='right'),
             Interval(-0.0642, -0.0512, closed='right')]}


In [92]:
diabetes.groupby("bmi_cut")["weights"].sum() / diabetes["weights"].sum()

bmi_cut
(-0.0905, -0.0772]    0.037523
(-0.0772, -0.0642]    0.082552
(-0.0642, -0.0512]    0.105066
(-0.0512, -0.0381]    0.142589
(-0.0381, -0.0251]    0.183865
(-0.0251, -0.012]     0.183865
(-0.012, 0.00102]     0.191745
(0.00102, 0.0141]     0.014259
(0.0141, 0.0271]      0.013508
(0.0271, 0.0401]      0.009381
(0.0401, 0.0532]      0.009381
(0.0532, 0.0662]      0.010507
(0.0662, 0.0793]      0.006004
(0.0793, 0.0923]      0.002627
(0.0923, 0.105]       0.003377
(0.105, 0.118]        0.001126
(0.118, 0.131]        0.001501
(0.131, 0.144]        0.000375
(0.144, 0.158]        0.000000
(0.158, 0.171]        0.000750
Name: weights, dtype: float64

## Unseen level handling

By default, categorical levels in new data that were not seen in the training data will be mapped to rare. This is not always desirable, for example there may be unseen level handling steps for this field futher down the pipeline. Setting the `encode_unseen_levels` parameter to `False` (defualt `True`) will cause unseen levels to be left unchanged.

In [93]:
grp_no_rare_encoding = GroupRareLevelsTransformer(
    columns=["bmi_cut"],
    cut_off_percent=0.10,
    copy=True,
    verbose=False,
    encode_unseen_levels=False,
)

Lets train the transformer on a dataset will all of instances of one level removed.

In [94]:
level_to_remove = diabetes["bmi_cut"][0]
print(f"Removed interval: {level_to_remove}")
diabetes_incomplete = diabetes[diabetes["bmi_cut"] != diabetes["bmi_cut"][0]]

grp_no_rare_encoding.fit(diabetes_incomplete)

Removed interval: (0.0532, 0.0662]


GroupRareLevelsTransformer(columns=['bmi_cut'], cut_off_percent=0.1,
                           encode_unseen_levels=False)

For comparison, lets also refit the first transformer (with `encode_unseen_levels` set to `True`) on the incomplete dataset.

In [95]:
grp_1.fit(diabetes_incomplete)

BaseTransformer.fit() called


GroupRareLevelsTransformer(columns=['bmi_cut'], cut_off_percent=0.1)

If we transform the full dataset with the old transformer we will see that the interval not present in the training data is mapped to rare: the category has count zero in the transformed data.

In [96]:
grp_1.transform(diabetes)["bmi_cut"].value_counts()

BaseTransformer.transform() called


rare                  292
(-0.012, 0.00102]      52
(-0.0381, -0.0251]     49
(-0.0251, -0.012]      49
(0.0662, 0.0793]        0
(0.158, 0.171]          0
(0.144, 0.158]          0
(0.131, 0.144]          0
(0.118, 0.131]          0
(0.105, 0.118]          0
(0.0923, 0.105]         0
(0.0793, 0.0923]        0
(-0.0905, -0.0772]      0
(0.0532, 0.0662]        0
(-0.0772, -0.0642]      0
(0.0271, 0.0401]        0
(0.0141, 0.0271]        0
(0.00102, 0.0141]       0
(-0.0512, -0.0381]      0
(-0.0642, -0.0512]      0
(0.0401, 0.0532]        0
Name: bmi_cut, dtype: int64

Now if we transform the full dataset with the grp_no_rare_encoding transformer, we'll see this interval is left alone. There are 28 counts present in the transformer dataset.

In [97]:
grp_no_rare_encoding.transform(diabetes)["bmi_cut"].value_counts()

rare                  264
(-0.012, 0.00102]      52
(-0.0381, -0.0251]     49
(-0.0251, -0.012]      49
(0.0532, 0.0662]       28
(0.0793, 0.0923]        0
(0.158, 0.171]          0
(0.144, 0.158]          0
(0.131, 0.144]          0
(0.118, 0.131]          0
(0.105, 0.118]          0
(0.0923, 0.105]         0
(-0.0905, -0.0772]      0
(0.0662, 0.0793]        0
(-0.0772, -0.0642]      0
(0.0271, 0.0401]        0
(0.0141, 0.0271]        0
(0.00102, 0.0141]       0
(-0.0512, -0.0381]      0
(-0.0642, -0.0512]      0
(0.0401, 0.0532]        0
Name: bmi_cut, dtype: int64